In [103]:
import numpy as np
import pandas as pd

In [104]:
'''
def funcion_prioridad_algoles(prioridad='almacen'):
    if prioridad == 'almacen':
        algo4()
        algo5()
    elif prioridad == 'produccion':
        algo5()
        algo4()
    else:
        print('No existe esa prioridad. Favor hablar con Ale')
    return
'''

"\ndef funcion_prioridad_algoles(prioridad='almacen'):\n    if prioridad == 'almacen':\n        algo4()\n        algo5()\n    elif prioridad == 'produccion':\n        algo5()\n        algo4()\n    else:\n        print('No existe esa prioridad. Favor hablar con Ale')\n    return\n"

In [105]:
#temporada: 1 es verano, 0 es invierno.
inv_verano = pd.read_excel('../02_data/verano.xlsx', sheet_name='veranos', skiprows=1)
inv_verano = pd.melt(inv_verano, id_vars=['zona'], value_vars=inv_verano.columns[1:] , var_name='mes_no', value_name='temporada')


import datetime
import dateutil.relativedelta as relativedelta

mes_inicio = 9
año_inicio = 2023

date_inicio = datetime.datetime(year=año_inicio, month=mes_inicio, day=10)
primer_periodo = str(date_inicio.year) + '-' + str(date_inicio.month).zfill(2)

periodos = []
horizonte_planificacion = 12
for mes in range(horizonte_planificacion + 1):
    aux_per = date_inicio + relativedelta.relativedelta(months=mes)
    periodos.append(str(aux_per.year) + '-' + str(aux_per.month).zfill(2))

periodos = pd.DataFrame(periodos, columns=['mes_real'])
periodos['año'] = periodos['mes_real'].str[:4]
periodos['mes_no'] = periodos['mes_real'].str[5:7].astype(int)
periodos['mes_planificado'] = periodos.index + 1
periodos = pd.merge(periodos, inv_verano, how='left', on='mes_no')
periodos.head()

,mes_real,año,mes_no,mes_planificado,zona,temporada
0,2023-09,2023,9,1,chillan,0
1,2023-09,2023,9,1,constitucion,0
2,2023-09,2023,9,1,arauco,0
3,2023-09,2023,9,1,valdivia,0
4,2023-10,2023,10,2,chillan,1


In [106]:
rango_mes_tipo = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='rango_mes_tipo')
dict_tiporango = rango_mes_tipo.set_index(['mes_planificado', 'nombre_planta', 'producto'])['id_tipo_rango'].to_dict()

rangos_demanda = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='parametro_antiguedad')

rangos_demanda_ext = pd.DataFrame(columns=['id_tipo_rango', 'rango', 'antiguedad'])
for ix in rangos_demanda.index:
    for ant in range(rangos_demanda.at[ix, 'inicio_rango'], rangos_demanda.at[ix, 'fin_rango'] + 1):
        aux = [[rangos_demanda.at[ix, 'id_tipo_rango'], rangos_demanda.at[ix, 'rango'], ant ]]
        aux = pd.DataFrame(aux, columns=['id_tipo_rango', 'rango', 'antiguedad'])
        rangos_demanda_ext = pd.concat([rangos_demanda_ext, aux], ignore_index=True)

rangos_demanda_ext['antiguedad'] = rangos_demanda_ext['antiguedad'].astype(np.int64).astype(str)

In [112]:
#Lectura de parametros planta
parametros_planta = pd.read_excel('../02_data/plantas.xlsx', sheet_name='parametros_planta')
parametros_planta['stock_ideal_planta'] = parametros_planta['ideal_dias_autonomia'] * parametros_planta['consumo_diario'] 

#Lectura consumo plantas
consumo_plantas = pd.read_excel('../02_data/plantas.xlsx', sheet_name='consumo_plantas')
consumo_plantas = pd.melt(consumo_plantas, consumo_plantas.columns[:2], consumo_plantas.columns[2:], var_name='mes_planificado', value_name='volumen_consumo')

## Matriz de demandas.

#Consumo_planta_tipo + parametros_planta_tipo
#Calculo demanda
df = pd.merge(consumo_plantas, parametros_planta, on=['nombre_planta', 'producto'])
df.loc[df['mes_planificado'] != 1, 'stock_inicial'] = 0
df['demanda'] = df['volumen_consumo'] + df['stock_ideal_planta'] - df['stock_inicial']

#Agregar los tipos de rango por mes
df = pd.merge(df, rango_mes_tipo, how='left', on=['nombre_planta', 'producto', 'mes_planificado'])
df = pd.merge(df, rangos_demanda, how='left', on='id_tipo_rango')
df['demanda_rango'] = df['demanda'] * df['porcentaje']
df = df[['nombre_planta', 'producto', 'mes_planificado', 'id_tipo_rango', 'rango', 'inicio_rango', 'fin_rango', 'demanda_rango']].dropna()
df['id_tipo_rango'] = df['id_tipo_rango'].astype(int)
df['inicio_rango'] = df['inicio_rango'].astype(int)
df['fin_rango'] = df['fin_rango'].astype(int)

demanda_plantas = df.copy()
demanda_plantas.to_csv('../03_pydata/demanda_plantas.csv', index=False, sep=';', decimal=',')

In [99]:
#Carga Ingresos: Compras y produccion
base_ingresos = pd.read_excel('../02_data/ingresos.xlsx', sheet_name='ingresos')
display(base_ingresos.head())

#Temporada de ingresos
dict_temp_ingresos = pd.merge(base_ingresos, periodos, how='left', on='zona')
dict_temp_ingresos = dict_temp_ingresos[['id_ingreso', 'mes_planificado', 'temporada']]
dict_temp_ingresos = dict_temp_ingresos.set_index(['id_ingreso', 'mes_planificado'])['temporada'].to_dict()

#Carga Ingreso: Volumen mensual
vol_ingresos = pd.read_excel('../02_data/ingresos_volumen.xlsx', sheet_name='ingresovolumen', skiprows=1)
vol_ingresos = pd.melt(vol_ingresos, vol_ingresos.columns[:1], vol_ingresos.columns[1:], var_name='mes_planificado', value_name='volumen')
vol_ingresos = pd.merge(base_ingresos, vol_ingresos, on='id_ingreso', how='left')
vol_ingresos.to_csv('../03_pydata/ingresos_volumen.csv', sep=';', decimal=',', index=False)

,id_ingreso,nombre_ingreso,nombre_planta,producto,zona,costo existencia,costo transporte,tipo_ingreso,almacen_verano,almacen_invierno,tipo_almacen,cosecha_compra
0,1,produccion EUNI SC constitucion,planta1,EUNI SC,constitucion,10,20,directo_planta,N_A,N_A,bosque,cosecha
1,2,produccion EUNI arauco sc,planta1,EUNI SC,arauco,10,20,directo_planta,N_A,N_A,bosque,cosecha
2,3,produccion EUNI valdivia sc,planta1,EUNI SC,valdivia,10,20,directo_planta,N_A,N_A,bosque,cosecha
3,4,produccion EUGL constitucion sc,planta1,EUGL SC,constitucion,10,20,directo_planta,N_A,N_A,bosque,cosecha
4,5,produccion EUGL arauco sc,planta1,EUGL SC,arauco,10,20,directo_planta,N_A,N_A,bosque,cosecha


In [100]:
### Carga de Almacenes ###
base_almacenes = pd.read_excel('../02_data/almacenes.xlsx', sheet_name='almacenes')
no_almacenes = len(base_almacenes)

alm_mes_temp = base_almacenes[['id_almacen', 'zona', 'carpeta']].copy().drop_duplicates(ignore_index=True)
alm_mes_temp = pd.merge(alm_mes_temp, periodos, how='left', on='zona')
alm_mes_temp = alm_mes_temp[['id_almacen', 'carpeta', 'mes_real', 'zona', 'mes_planificado', 'temporada']].copy().drop_duplicates(ignore_index=True)
alm_mes_temp['bloqueo'] = False
alm_mes_temp.loc[(alm_mes_temp['carpeta'] == 'tierra') & (alm_mes_temp['temporada'] == 0), 'bloqueo'] = True
dict_temp_almacen = alm_mes_temp.set_index(['id_almacen', 'mes_planificado'])['bloqueo'].to_dict()

#Stock almacen EN MES 0
stock_almacen = pd.read_excel('../02_data/almacenes_stock.xlsx', sheet_name='cierrestock', skiprows=1)
stock_almacen = pd.melt(stock_almacen, stock_almacen.columns[:1], stock_almacen.columns[1:], var_name='antiguedad', value_name='stock_inicial').fillna(0)

stock_almacen = pd.merge(base_almacenes, stock_almacen, how='left', on='id_almacen')
stock_almacen['antiguedad'] = stock_almacen['antiguedad'].astype(np.int64).astype(str)
stock_almacen.insert(loc=len(stock_almacen.columns)-1, column='mes_planificado', value=1)


dict_alm_acanche = stock_almacen[['id_almacen', 'id_almacen_acanche']].drop_duplicates().set_index('id_almacen')['id_almacen_acanche'].to_dict()

In [101]:
periodos
rango_mes_tipo
dict_tiporango
rangos_demanda
rangos_demanda_ext
dict_temp_almacen
dict_temp_ingresos
dict_alm_acanche

stock_almacen
vol_ingresos
demanda_plantas

,nombre_planta,producto,mes_planificado,id_tipo_rango,rango,inicio_rango,fin_rango,demanda_rango
36,planta1,EUGL CC,1,1,0-3,0,3,57.533
37,planta1,EUGL CC,1,1,4-6,4,6,16.438
38,planta1,EUGL CC,1,1,7-9,7,9,8.219
39,planta1,EUGL CC,2,2,0-3,0,3,56.413
40,planta1,EUGL CC,2,2,4-6,4,6,8.059
41,planta1,EUGL CC,2,2,7-13,7,13,16.118
42,planta1,EUGL CC,3,3,0-3,0,3,65.154
43,planta1,EUGL CC,3,3,4-6,4,6,21.718
44,planta1,EUGL CC,3,3,7-9,7,9,10.859
45,planta1,EUGL CC,3,3,10-13,10,13,10.859


In [102]:
demanda_plantas

,nombre_planta,producto,mes_planificado,id_tipo_rango,rango,inicio_rango,fin_rango,demanda_rango
36,planta1,EUGL CC,1,1,0-3,0,3,57.533
37,planta1,EUGL CC,1,1,4-6,4,6,16.438
38,planta1,EUGL CC,1,1,7-9,7,9,8.219
39,planta1,EUGL CC,2,2,0-3,0,3,56.413
40,planta1,EUGL CC,2,2,4-6,4,6,8.059
41,planta1,EUGL CC,2,2,7-13,7,13,16.118
42,planta1,EUGL CC,3,3,0-3,0,3,65.154
43,planta1,EUGL CC,3,3,4-6,4,6,21.718
44,planta1,EUGL CC,3,3,7-9,7,9,10.859
45,planta1,EUGL CC,3,3,10-13,10,13,10.859


In [71]:
tabla_demanda = pd.DataFrame()


col_tabla_disponibilidad = ['mes', 'origen', 'producto', 'antiguedad', 'stock', 'volumen']
tabla_disponibilidad = pd.DataFrame(columns=col_tabla_disponibilidad)

col_tabla_movimiento = ['mes', 'origen', 'destino', 'producto', 'antiguedad', 'volumen']
tabla_movimiento = pd.DataFrame(columns=col_tabla_movimiento)

def agregar_movimiento(mes, origen, destino, producto, antiguedad, volumen):
    fila = [mes, origen, destino, producto, antiguedad, volumen]
    fila = pd.DataFrame([fila], columns=tabla_movimiento.columns)
    return pd.concat([tabla_movimiento, fila], ignore_index=True)


for mes in [1]:
    for planta in demanda_plantas['nombre_planta'].unique():
        for producto in demanda_plantas[demanda_plantas['nombre_planta'] == planta]['producto'].unique():
        # for producto in ['EUGL CC']:
            print(planta, producto, mes)

            tipo_rango = dict_tiporango[(mes, planta, producto)]
            rde = rangos_demanda_ext.copy()
            rde = rde.loc[rde['id_tipo_rango'] == tipo_rango].drop(columns=['id_tipo_rango'])

            #Demanda del mes
            dpla = demanda_plantas.copy()
            dpla = dpla[(dpla['nombre_planta'] == planta) & (dpla['producto'] == producto) & (dpla['mes_planificado'] == mes)]

            #Almacenes disponibles
            alm_disponible = alm_mes_temp.copy()
            alm_disponible = alm_disponible[(alm_disponible['mes_planificado'] == mes) & (alm_disponible['bloqueo'] == False)]
            # alm_disponible = alm_disponible[(alm_disponible['mes_planificado'] == mes)]
            alm_disponible = list(alm_disponible['id_almacen'].unique())

            #Almacenes con rango 
            alma = stock_almacen.copy()
            alma = alma[(alma['id_almacen'].isin(alm_disponible))
                        & (alma['nombre_planta'] == planta) 
                        & (alma['producto'] == producto) 
                        & (alma['mes_planificado'] == mes)]
            alma = pd.merge(alma, rde, how='left', on=['antiguedad'])

            if len(alma[alma['rango'].isnull()]) > 0:
                print('ERROR: las siguientes antiguedades no tienen un rango en el "tipo_rango" ')
                print(alma[alma['rango'].isnull()]['antiguedad'].unique())

            #ingdir: ingresos directos
            ingdir = vol_ingresos.copy()
            ingdir = ingdir.loc[(ingdir['nombre_planta'] == planta) 
                            & (ingdir['producto'] == producto) 
                            & (ingdir['mes_planificado'] == mes)
                            & (ingdir['tipo_ingreso'] == 'directo_planta')]
            
            #inglib: ingresos libres
            inglib = vol_ingresos.copy()
            inglib = inglib.loc[(inglib['nombre_planta'] == planta) 
                & (inglib['producto'] == producto) 
                & (inglib['mes_planificado'] == mes)
                & (inglib['tipo_ingreso'] == 'libre')]
            
          

            print('Algo 1: Modificacion de demandas. Desde lo mas antiguo a lo mas nuevo')
            #Si la planta no tiene demanda en el mes actual, se continua el ciclo.
            if len(dpla) > 0:
                dpla = dpla.sort_values(by='inicio_rango', ascending=False)
                
                alma_aux = alma.groupby(['nombre_planta', 'producto', 'rango'], as_index=False).agg(volumen=('stock_inicial', 'sum'))
                alma_aux = alma_aux.sort_values(by='rango', ascending=False).reset_index(drop=True)

                dpla = pd.merge(dpla, alma_aux, how='left', on=['nombre_planta', 'producto', 'rango'])
                dpla['volumen'] = dpla['volumen'].fillna(0)

                #Algoritmo Ale
                for ix in dpla.index[:-1]:
                    if dpla.at[ix, 'volumen'] < dpla.at[ix, 'demanda_rango']:
                        dpla.at[ix + 1, 'demanda_rango'] +=  dpla.at[ix, 'demanda_rango'] - dpla.at[ix, 'volumen']
                        dpla.at[ix, 'demanda_rango'] = dpla.at[ix, 'volumen']
                dpla = dpla.drop(columns=['volumen']).sort_values(by=['mes_planificado', 'inicio_rango'])





            print('Algo 2: Modificacion de demandas. El ingreso directo debe estar complemente incluido en la demanda del primer rango')
            dpla = dpla.sort_values(by='inicio_rango', ascending=True).reset_index(drop=True)
            vol_demanda = dpla['demanda_rango'].sum()
            
            #Se filtran los ingresos de planta directo.
            vol_fresco = 0
            aux_ingdir = ingdir.groupby(['nombre_planta', 'producto', 'mes_planificado'], as_index=False).agg({'volumen':'sum'})
            if len(aux_ingdir) > 0:
                #Indicadores
                vol_fresco = aux_ingdir.at[0, 'volumen']
                vol_stock = vol_fresco - vol_demanda
                if vol_stock < 0:
                    vol_stock = 0.0

                # print(f'vol_demanda: {vol_demanda}, vol_ingreso_directo: {vol_fresco}, vol_stock: {vol_stock}')

                #Algoritmo Ale
                if vol_fresco > dpla.at[0, 'demanda_rango']:
                    dpla.at[1, 'demanda_rango'] = dpla.at[1, 'demanda_rango'] + dpla.at[0, 'demanda_rango'] - vol_fresco
                    dpla.at[0, 'demanda_rango'] = vol_fresco

                for ix in dpla.index[1:-1]:
                    if dpla.at[ix, 'demanda_rango'] < 0:
                        dpla.at[ix+1, 'demanda_rango'] = dpla.at[ix+1, 'demanda_rango'] + dpla.at[ix, 'demanda_rango']
                        dpla.at[ix, 'demanda_rango'] = 0
              


                
            print('Algo 3: Venta de ingresos directos')
            #Ingresos_directos -> planta
            for ix in ingdir.index:
                tabla_movimiento = agregar_movimiento(mes=mes,
                                                      origen=ingdir.at[ix, 'id_ingreso'],
                                                      destino=planta,
                                                      producto=producto,
                                                      antiguedad=0,
                                                      volumen=ingdir.at[ix,'volumen'])

            demanda_faltante = dpla.at[0, 'demanda_rango'] - vol_fresco
            print(f'... A3 demanda faltante: {demanda_faltante}')



            print('Algo 4: Venta de ingresos libres')
            #De ingresos libres -> planta y almacen.
            for ix in inglib.index:
                #Identificacion de almacen de destino
                tempo_ingreso_actual = dict_temp_ingresos[(inglib.at[ix, 'id_ingreso'], mes)]
                tempo_ingreso_siguiente = dict_temp_ingresos[(inglib.at[ix, 'id_ingreso'], mes+1)]

                if tempo_ingreso_actual == 1 and tempo_ingreso_siguiente == 0:
                    almacen_ingreso = inglib.at[ix, 'almacen_verano']
                    almacen_ingreso = dict_alm_acanche[almacen_ingreso]
                elif tempo_ingreso_actual == 1:
                    almacen_ingreso = inglib.at[ix, 'almacen_verano']
                elif tempo_ingreso_actual == 0:
                    almacen_ingreso = inglib.at[ix, 'almacen_invierno']
                else:
                    raise NotImplementedError('A4-1. Existe un caso no considerado en el ciclo')


                if demanda_faltante >= inglib.at[ix, 'volumen']:
                    tabla_movimiento = agregar_movimiento(mes=mes,
                                                          origen=inglib.at[ix, 'id_ingreso'],
                                                          destino=planta,
                                                          producto=producto,
                                                          antiguedad=0,
                                                          volumen=inglib.at[ix, 'volumen'])
                    demanda_faltante = demanda_faltante - inglib.at[ix, 'volumen']

                elif demanda_faltante > 0 and inglib.at[ix, 'volumen'] > demanda_faltante:
                    tabla_movimiento = agregar_movimiento(mes=mes,
                                                          origen=inglib.at[ix, 'id_ingreso'],
                                                          destino=planta,
                                                          producto=producto,
                                                          antiguedad=0,
                                                          volumen=demanda_faltante)                    

                    tabla_movimiento = agregar_movimiento(mes=mes,
                                                          origen=inglib.at[ix, 'id_ingreso'],
                                                          destino=almacen_ingreso,
                                                          producto=producto,
                                                          antiguedad=0,
                                                          volumen=inglib.at[ix, 'volumen'] - demanda_faltante)
                    
                    demanda_faltante = 0

                elif demanda_faltante == 0:
                    tabla_movimiento = agregar_movimiento(mes=mes,
                                                          origen=inglib.at[ix, 'id_ingreso'],
                                                          destino=almacen_ingreso,
                                                          producto=producto,
                                                          antiguedad=0,
                                                          volumen=inglib.at[ix, 'volumen'])
                
                else:
                    raise NotImplementedError('A4-2. Existe un caso no considerado en el ciclo')

    



            print('Algo 5: Se completa demanda faltante en primer rango (0-3) desde almacenes')
            #Almacen -> Plata | Almacen -> Almacen

            alma = alma.sort_values(by=['prioridad', 'antiguedad'], ascending=[True, False])
            #Solo se itera sobre el primer rango.
            aux_rangos = rangos_demanda[rangos_demanda['id_tipo_rango'] == 1].reset_index(drop=True)
            aux_rangos = np.arange(aux_rangos.at[0, 'fin_rango'], aux_rangos.at[0, 'inicio_rango']-1, -1)
            aux_rangos


            for r in aux_rangos:
                for ix in alma.index:
                    tempo_almacen_actual = dict_temp_almacen[(alma.at[ix, 'id_almacen'], mes)]
                    tempo_almacen_siguiente = dict_temp_almacen[(alma.at[ix, 'id_almacen'], mes+1)]

                    if alma.at[ix,'carpeta'] == 'tierra' and tempo_almacen_actual == 1 and tempo_almacen_siguiente == 0:
                        almacen_destino = alma.at[ix,'id_almacen_acanche']
                    else:
                        almacen_destino = alma.at[ix,'id_almacen']
        

                    if alma.at[ix,'antiguedad'] == str(r):
                        if demanda_faltante >= alma.at[ix, 'stock_inicial']:
                            tabla_movimiento = agregar_movimiento(mes=mes,
                                                                  origen=alma.at[ix,'id_almacen'],
                                                                  destino=planta,
                                                                  producto=producto,
                                                                  antiguedad=alma.at[ix,'antiguedad'],
                                                                  volumen=alma.at[ix, 'stock_inicial'])
                            demanda_faltante = demanda_faltante - alma.at[ix, 'stock_inicial']

                        elif demanda_faltante > 0 and alma.at[ix, 'stock_inicial'] > demanda_faltante:
                            #Movimiento Almacen-Planta
                            tabla_movimiento = agregar_movimiento(mes=mes,
                                                                  origen=alma.at[ix,'id_almacen'],
                                                                  destino=planta,
                                                                  producto=producto,
                                                                  antiguedad=alma.at[ix,'antiguedad'],
                                                                  volumen=demanda_faltante)
                            #Movimiento Almacen->Almacen-Acanche
                            tabla_movimiento = agregar_movimiento(mes=mes,
                                                                  origen=alma.at[ix,'id_almacen'],
                                                                  destino=almacen_destino,
                                                                  producto=producto,
                                                                  antiguedad=alma.at[ix,'antiguedad'],
                                                                  volumen=alma.at[ix, 'stock_inicial'] - demanda_faltante)
                            demanda_faltante = 0
                        
                        elif demanda_faltante == 0:
                            tabla_movimiento = agregar_movimiento(mes=mes,
                                                                  origen=alma.at[ix,'id_almacen'],
                                                                  destino=almacen_destino,
                                                                  producto=producto,
                                                                  antiguedad=alma.at[ix,'antiguedad'],
                                                                  volumen=alma.at[ix, 'stock_inicial'])
                             


            print(f'... A5 demanda_faltante: {demanda_faltante}')
            #Se prodria cambiar el orden entre algo4 y algo5


            print('Algo 6: Se completa la demanda faltante para el resto de los rangos (4-6 en adelante)')
            aux_rangos = rangos_demanda[rangos_demanda['id_tipo_rango'] == 1].reset_index(drop=True)
            for ix in aux_rangos.index[1:]:
                rango_actual = aux_rangos.at[ix, 'rango']
                demanda_faltante += dpla.set_index('rango').at[rango_actual, 'demanda_rango']
                print(f'Inicio ciclo: rango {rango_actual}, demanda faltante {demanda_faltante}')
                it_rangos = np.arange(aux_rangos.at[ix, 'fin_rango'], aux_rangos.at[ix, 'inicio_rango']-1, -1)

            for r in it_rangos:
                for ix in alma.index:
                    tempo_almacen_actual = dict_temp_almacen[(alma.at[ix, 'id_almacen'], mes)]
                    tempo_almacen_siguiente = dict_temp_almacen[(alma.at[ix, 'id_almacen'], mes+1)]

                    if alma.at[ix,'carpeta'] == 'tierra' and tempo_almacen_actual == 1 and tempo_almacen_siguiente == 0:
                        almacen_destino = alma.at[ix,'id_almacen_acanche']
                    else:
                        almacen_destino = alma.at[ix,'id_almacen']
        

                    if alma.at[ix,'antiguedad'] == str(r):
                        if demanda_faltante >= alma.at[ix, 'stock_inicial']:
                            tabla_movimiento = agregar_movimiento(mes=mes,
                                                                  origen=alma.at[ix,'id_almacen'],
                                                                  destino=planta,
                                                                  producto=producto,
                                                                  antiguedad=alma.at[ix,'antiguedad'],
                                                                  volumen=alma.at[ix, 'stock_inicial'])
                            demanda_faltante = demanda_faltante - alma.at[ix, 'stock_inicial']

                        elif demanda_faltante > 0 and alma.at[ix, 'stock_inicial'] > demanda_faltante:
                            #Movimiento Almacen-Planta
                            tabla_movimiento = agregar_movimiento(mes=mes,
                                                                  origen=alma.at[ix,'id_almacen'],
                                                                  destino=planta,
                                                                  producto=producto,
                                                                  antiguedad=alma.at[ix,'antiguedad'],
                                                                  volumen=demanda_faltante)
                            #Movimiento Almacen->Almacen-Acanche
                            tabla_movimiento = agregar_movimiento(mes=mes,
                                                                  origen=alma.at[ix,'id_almacen'],
                                                                  destino=almacen_destino,
                                                                  producto=producto,
                                                                  antiguedad=alma.at[ix,'antiguedad'],
                                                                  volumen=alma.at[ix, 'stock_inicial'] - demanda_faltante)
                            demanda_faltante = 0
                        
                        elif demanda_faltante == 0:
                            tabla_movimiento = agregar_movimiento(mes=mes,
                                                                  origen=alma.at[ix,'id_almacen'],
                                                                  destino=almacen_destino,
                                                                  producto=producto,
                                                                  antiguedad=alma.at[ix,'antiguedad'],
                                                                  volumen=alma.at[ix, 'stock_inicial'])




planta1 EUGL CC 1
ERROR: las siguientes antiguedades no tienen un rango en el "tipo_rango" 
['10' '11' '12' '13']
Algo 1: Modificacion de demandas. Desde lo mas antiguo a lo mas nuevo
Algo 2: Modificacion de demandas. El ingreso directo debe estar complemente incluido en la demanda del primer rango
Algo 3: Venta de ingresos directos
... A3 demanda faltante: 0.0
Algo 4: Venta de ingresos libres
Algo 5: Se completa demanda faltante en primer rango (0-3) desde almacenes
... A5 demanda_faltante: 0.0
Algo 6: Se completa la demanda faltante para el resto de los rangos (4-6 en adelante)
Inicio ciclo: rango 4-6, demanda faltante 0.0
Inicio ciclo: rango 7-9, demanda faltante 2.189999999999989


In [72]:
tabla_movimiento

,mes,origen,destino,producto,antiguedad,volumen
0,1,10,planta1,EUGL CC,0,20.00
1,1,11,planta1,EUGL CC,0,20.00
2,1,12,planta1,EUGL CC,0,20.00
3,1,16,planta1,EUGL CC,0,20.00
4,1,28,alm22,EUGL CC,0,20.00
5,1,alm22,alm22,EUGL CC,3,1.00
6,1,alm3,alm3,EUGL CC,3,1.00
7,1,alm6,alm6,EUGL CC,3,1.00
8,1,alm22,alm22,EUGL CC,2,1.00
9,1,alm3,alm3,EUGL CC,2,1.00


In [73]:
entradas_producto = tabla_movimiento.groupby(['mes', 'destino', 'producto', 'antiguedad'], as_index=False).agg(entrada_volumen=('volumen','sum'))
salidas_producto = tabla_movimiento.groupby(['mes', 'origen', 'producto', 'antiguedad'], as_index=False).agg(salida_volumen=('volumen','sum'))